In [1]:
import pandas as pd

# Create FP&A-style sample data
df = pd.DataFrame({
    "month": ["2025-07","2025-07","2025-07","2025-07",
              "2025-08","2025-08","2025-08","2025-08"],
    "cost_center": ["IT","IT","Marketing","Operations",
                    "IT","IT","Marketing","Operations"],
    "category": ["Payroll","Software","Campaigns","Travel",
                 "Payroll","Software","Campaigns","Travel"],
    "budget": [50000,12000,18000,4000,
               51000,12000,20000,4500],
    "actual": [52000,15000,21000,3000,
               50500,11800,19500,6000]
})

df


,month,cost_center,category,budget,actual
0,2025-07,IT,Payroll,50000,52000
1,2025-07,IT,Software,12000,15000
2,2025-07,Marketing,Campaigns,18000,21000
3,2025-07,Operations,Travel,4000,3000
4,2025-08,IT,Payroll,51000,50500
5,2025-08,IT,Software,12000,11800
6,2025-08,Marketing,Campaigns,20000,19500
7,2025-08,Operations,Travel,4500,6000


In [3]:
# Create simple executive-style explanations for overspends

# Calculate variance and percentage variance
df["variance"] = df["actual"] - df["budget"]
df["variance_pct"] = (df["variance"] / df["budget"]) * 100

# Filter for overspends
overspends = df[df["variance"] > 0].copy()

def explain_overspend(row):
    return (
        f"In {row['month']}, {row['category']} spend in the {row['cost_center']} cost center "
        f"exceeded budget by ${row['variance']:,} "
        f"({row['variance_pct']:.1f}%)."
    )

overspends["explanation"] = overspends.apply(explain_overspend, axis=1)

overspends[["month", "cost_center", "category", "explanation"]]

,month,cost_center,category,explanation
0,2025-07,IT,Payroll,"In 2025-07, Payroll spend in the IT cost cente..."
1,2025-07,IT,Software,"In 2025-07, Software spend in the IT cost cent..."
2,2025-07,Marketing,Campaigns,"In 2025-07, Campaigns spend in the Marketing c..."
7,2025-08,Operations,Travel,"In 2025-08, Travel spend in the Operations cos..."


In [4]:
# FP&A calculations
df["variance"] = df["actual"] - df["budget"]
df["variance_pct"] = (df["variance"] / df["budget"]) * 100

# Overspends only (actual > budget)
overspends = df[df["variance"] > 0].copy().sort_values("variance", ascending=False)

overspends


,month,cost_center,category,budget,actual,variance,variance_pct
1,2025-07,IT,Software,12000,15000,3000,25.000000
2,2025-07,Marketing,Campaigns,18000,21000,3000,16.666667
0,2025-07,IT,Payroll,50000,52000,2000,4.000000
7,2025-08,Operations,Travel,4500,6000,1500,33.333333


In [5]:
# Executive-style commentary
def explain_overspend(row):
    return (
        f"In {row['month']}, {row['category']} spend in {row['cost_center']} exceeded budget by "
        f"${row['variance']:,} ({row['variance_pct']:.1f}%)."
    )

overspends["explanation"] = overspends.apply(explain_overspend, axis=1)

overspends[["month", "cost_center", "category", "variance", "variance_pct", "explanation"]]


,month,cost_center,category,variance,variance_pct,explanation
1,2025-07,IT,Software,3000,25.000000,"In 2025-07, Software spend in IT exceeded budg..."
2,2025-07,Marketing,Campaigns,3000,16.666667,"In 2025-07, Campaigns spend in Marketing excee..."
0,2025-07,IT,Payroll,2000,4.000000,"In 2025-07, Payroll spend in IT exceeded budge..."
7,2025-08,Operations,Travel,1500,33.333333,"In 2025-08, Travel spend in Operations exceede..."


In [6]:
# Monthly executive summary (auto)
total_var = df["variance"].sum()
top3 = overspends.head(3)[["month", "cost_center", "category", "variance"]]

summary = []
summary.append(f"Overall, total spend variance is ${total_var:,.0f} vs budget across the dataset.")

summary.append("Top overspend drivers:")
for _, r in top3.iterrows():
    summary.append(f"- {r['month']} | {r['cost_center']} | {r['category']}: ${r['variance']:,.0f} over budget")

print("\n".join(summary))


Overall, total spend variance is $7,300 vs budget across the dataset.
Top overspend drivers:
- 2025-07 | IT | Software: $3,000 over budget
- 2025-07 | Marketing | Campaigns: $3,000 over budget
- 2025-07 | IT | Payroll: $2,000 over budget
